In [ ]:
from src.universe.base import Universe

universe = Universe(n_customers=20, rounds_per_cycle=365)
output = universe.generate_orders(n_cycles=3)

In [2]:
output.contract_ammendment.sum() / (output.shape[0] / 50)

np.float64(0.1972940486241289)

In [3]:
output.new_customer.sum() / (output.shape[0] / 50)

0.01064278866104775

In [4]:
output

,cycle,round,customer_id,order_number,item_category_id,service_id,variant,price,quantity,final_price,new_customer,contract_ammendment,1_price,2_price,2_probability,3_price,4_price
0,0,1,1.0,1,1.0,1_10,None,802.12,1.0,802.120000,True,False,False,False,False,False,False
1,0,1,1.0,1,2.0,2_7,0,35.29,1.0,30.449284,True,False,False,False,True,False,False
1,0,1,1.0,1,2.0,2_7,0,35.29,1.0,30.449284,True,False,False,True,False,False,False
2,0,1,1.0,1,2.0,2_13,0,44.59,1.0,38.473607,True,False,False,False,True,False,False
2,0,1,1.0,1,2.0,2_13,0,44.59,1.0,38.473607,True,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481144,3,365,50.0,3,1.0,1_2,None,463.20,1.0,200.919444,False,False,True,False,False,False,False
481145,3,365,50.0,3,2.0,2_7,0,29.28,1.0,17.701905,False,False,False,False,True,False,False
481145,3,365,50.0,3,2.0,2_7,0,29.28,1.0,17.701905,False,False,False,True,False,False,False
481146,3,365,50.0,4,1.0,1_4,None,1817.36,1.0,1817.360000,False,False,False,False,False,False,False


In [5]:
output.columns

Index(['cycle', 'round', 'customer_id', 'order_number', 'item_category_id',
       'service_id', 'variant', 'price', 'quantity', 'final_price',
       'new_customer', 'contract_ammendment', '1_price', '2_price',
       '2_probability', '3_price', '4_price'],
      dtype='object')